# 🤖 TEMPLATE — MODELING NOTEBOOK

## Objetivo:

Treinar, avaliar e comparar modelos de machine learning para resolver
o problema definido, garantindo generalização, reprodutibilidade
e alinhamento com métricas de negócio.

Critérios:
- Validação robusta
- Métricas adequadas
- Baseline claro

## 📚 1. Imports e Configurações

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import (
    train_test_split,
    StratifiedKFold,
    cross_validate
)

from sklearn.metrics import (
    roc_auc_score,
    f1_score,
    precision_score,
    recall_score,
    classification_report
)

from src.config import TARGET_COL, ID_COL, DATA_PATH_PROCESSED, TEST_SIZE, RANDOM_STATE, N_SPLITS

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier


from xgboost import XGBClassifier

import mlflow
import mlflow.sklearn

## 📂 2. Carregamento dos Dados

In [ ]:
df = pd.read_csv(DATA_PATH_PROCESSED)

X = df.drop(columns=[TARGET_COL, ID_COL], errors="ignore")
y = df[TARGET_COL]

## 🔀 3. Train / Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=TEST_SIZE,
    stratify=y,
    random_state=RANDOM_STATE
)


**📌 Por quê?**

- Hold-out final garante avaliação honesta
- Estratificação mantém proporção do target

## 📉 4. Baseline Model

In [ ]:
baseline_model = LogisticRegression(
    max_iter=1000,
    class_weight="balanced",
    random_state=RANDOM_STATE
)


In [ ]:
baseline_model.fit(X_train, y_train)

y_pred = baseline_model.predict(X_test)
y_proba = baseline_model.predict_proba(X_test)[:, 1]

print("ROC-AUC:", roc_auc_score(y_test, y_proba))
print(classification_report(y_test, y_pred))


📌 **INSIGHT ESPERADO**

Qualquer modelo mais complexo deve superar esse baseline.

## 🔁 5. Validação Cruzada

In [ ]:
cv = StratifiedKFold(
    n_splits=N_SPLITS,
    shuffle=True,
    random_state=RANDOM_STATE
)

scoring = {
    "roc_auc": "roc_auc",
    "f1": "f1",
    "precision": "precision",
    "recall": "recall"
}


## 🤖 6. Modelos Candidatos

In [ ]:
models = {
    "logistic_regression": LogisticRegression(
        max_iter=1000,
        class_weight="balanced",
        random_state=RANDOM_STATE
    ),
    "random_forest": RandomForestClassifier(
        n_estimators=300,
        random_state=RANDOM_STATE,
        class_weight="balanced"
    ),
    "xgboost": XGBClassifier(
        eval_metric="logloss",
        use_label_encoder=False,
        random_state=RANDOM_STATE
    ),
    "catboost": CatBoostClassifier(
        random_state=RANDOM_STATE,
        verbose=0
    ),
    "lightgbm": LGBMClassifier(
        random_state=RANDOM_STATE
    ),
    "knn": KNeighborsClassifier(),
    "svc": SVC(
        probability=True,
        random_state=RANDOM_STATE),
    "naive_bayes": GaussianNB(),
    "decision_tree": DecisionTreeClassifier(
        random_state=RANDOM_STATE,
        class_weight="balanced"),
    "gradient_boosting": GradientBoostingClassifier(
        random_state=RANDOM_STATE
    ),
    "adaboost": AdaBoostClassifier(
        random_state=RANDOM_STATE
    ),
    "random_forest": RandomForestClassifier(
        n_estimators=300,
        random_state=RANDOM_STATE,
        class_weight="balanced"
    )
}


## 📊 7. Avaliação com Cross-Validation

In [ ]:
results = {}

for name, model in models.items():
    cv_results = cross_validate(
        model,
        X_train,
        y_train,
        cv=cv,
        scoring=scoring,
        n_jobs=-1
    )
    
    results[name] = {
        metric: np.mean(scores)
        for metric, scores in cv_results.items()
        if metric.startswith("test")
    }

pd.DataFrame(results).T

## 📈 8. Treinamento Final + Avaliação Hold-out

In [ ]:
best_model = models["xgboost"]

best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:, 1]

print("ROC-AUC:", roc_auc_score(y_test, y_proba))
print(classification_report(y_test, y_pred))

## 🔥 9. Feature Importance

In [ ]:
importances = best_model.feature_importances_

pd.Series(importances, index=X.columns).sort_values(ascending=False).head(15)

📌 **Insight**

- Explica decisões e valida hipóteses da EDA.

## 🧪 10. MLflow Tracking

In [ ]:
mlflow.set_experiment("modeling_experiment")

with mlflow.start_run():
    mlflow.log_param("model_type", "xgboost")
    mlflow.log_metric("roc_auc", roc_auc_score(y_test, y_proba))
    
    mlflow.sklearn.log_model(best_model, "model")

## 🧠 11. Registro de Decisões

**Decisões:**
- Modelo final: XGBoost
- Métrica principal: ROC-AUC
- Validação: Stratified K-Fold
- **Motivação:**
    - Melhor trade-off bias/variance
    - Melhor captura de não linearidades

## 🚀 12. Próximos Passos

**Próximos passos:**
- Tuning de hiperparâmetros
- Threshold tuning
- Monitoramento em produção
- Testes de drift